<a href="https://colab.research.google.com/github/felipeabe/artificial-neural-network/blob/projetoComCSV/trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#uploading files
from google.colab import files
uploaded = files.upload()

#importing libs
import pandas as pd

Saving houses.csv to houses.csv


In [2]:

# importing multiclass dataset
dfm = pd.read_csv('obesity.csv')

#changing columns names
# Defining a dictionary with old column names as keys and new column names as values
column_mapping = {
    'family_history_with_overweight': 'family_overweight',
    'FAVC': 'caloric_food',
    'FCVC': 'eat_vegetables',
    'NCP': 'meals_day',
    'SMOKE': 'smoke',
    'CH2O': 'water_day',
    'SCC': 'track_calories',
    'FAF': 'physical_activity_week',
    'CALC': 'alcohol',
    'MTRANS': 'transportation_method',
    'NObeyesdad': 'obesity_level'
}

# Renaming the columns using the dictionary
dfm = dfm.rename(columns=column_mapping)


#Removing unnecessary columns
dfm = dfm.drop(columns=['CAEC', 'TUE'])

#encoding values (binary)
binary_mappings = {
    "Gender": {"Female": 0, "Male": 1},
    "family_overweight": {"yes": 1, "no": 0},
    "caloric_food": {"yes": 1, "no": 0},
    "smoke": {"yes": 1, "no": 0},
    "track_calories": {"yes": 1, "no": 0}
}

for col, mapping in binary_mappings.items():
    dfm[col] = dfm[col].map(mapping)

#encoding alcohol usage
encoding_map = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
dfm['alcohol'] = dfm['alcohol'].map(encoding_map)


#encoding transportation_method
custom_mapping = {
    "Walking": 0,
    "Bike": 0,
    "Motorbike": 1,
    "Automobile": 1,
    "Public_Transportation": 1
}
dfm['transportation_method'] = dfm['transportation_method'].map(custom_mapping)

#encoding output column (obesity_level)
obesity_mapping = {
    "Insufficient_Weight": 0,
    "Normal_Weight": 1,
    "Overweight_Level_I": 2,
    "Overweight_Level_II": 3,
    "Obesity_Type_I": 4,
    "Obesity_Type_II": 5,
    "Obesity_Type_III": 6
}
dfm['obesity_level'] = dfm ['obesity_level'].map(obesity_mapping)

#normalizing data
columns_to_normalize = ["Height", "Weight", "Age"]
dfm[columns_to_normalize] = dfm[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

#spliting the dataframe
#training 80% of dataset
#testing 20% of dataset
split_point = int(0.8 * len(dfm))
train_multiclass = dfm[:split_point]
test_multiclass = dfm[split_point:]


dfm.head(10)

,Gender,Age,Height,Weight,family_overweight,caloric_food,eat_vegetables,meals_day,smoke,water_day,track_calories,physical_activity_week,alcohol,transportation_method,obesity_level
0,0,0.149,0.321,0.187,1,0,2.0,3.0,0,2.0,0,0.0,0,1,1
1,0,0.149,0.132,0.127,1,0,3.0,3.0,1,3.0,1,3.0,1,1,1
2,1,0.191,0.660,0.284,1,0,2.0,3.0,0,2.0,0,2.0,2,1,1
3,1,0.277,0.660,0.358,0,0,3.0,3.0,0,2.0,0,2.0,2,0,2
4,1,0.170,0.623,0.379,0,0,2.0,1.0,0,2.0,0,0.0,1,1,3
5,1,0.319,0.321,0.104,0,1,2.0,3.0,0,2.0,0,0.0,1,1,1
6,0,0.191,0.094,0.119,1,1,3.0,3.0,0,2.0,0,1.0,1,1,1
7,1,0.170,0.358,0.104,0,0,2.0,3.0,0,2.0,0,3.0,1,1,1
8,1,0.213,0.623,0.187,1,1,3.0,3.0,0,2.0,0,1.0,2,1,1
9,1,0.170,0.509,0.216,1,1,2.0,3.0,0,2.0,0,1.0,0,1,1


In [3]:
#import binary dataset
dfb = pd.read_csv('alzheimer.csv')
#outcome:
# 1- alzheimer
# 0- no alzheimer

#selecting useful columns
selected_columns = ['Age', 'Gender','Smoking', 'EducationLevel', 'BMI' ,'SleepQuality', 'CholesterolLDL', 'FamilyHistoryAlzheimers','PhysicalActivity', 'Diagnosis']
dfb = dfb[selected_columns]

#normalizing data
columns_to_normalize = ["BMI", "SleepQuality", "Age", "CholesterolLDL", "FamilyHistoryAlzheimers", "PhysicalActivity"]
dfb[columns_to_normalize] = dfb[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

#spliting the dataframe
#training 80% of dataset
#testing 20% of dataset
split_point2 = int(0.8 * len(dfb))
train_binary = dfb[:split_point2]
test_binary = dfb[split_point2:]
train_binary.head(10)

,Age,Gender,Smoking,EducationLevel,BMI,SleepQuality,CholesterolLDL,FamilyHistoryAlzheimers,PhysicalActivity,Diagnosis
0,0.433,0,0,2,0.317,0.838,0.040,0.0,0.633,0
1,0.967,0,0,0,0.473,0.525,0.956,0.0,0.763,0
2,0.433,0,0,1,0.112,0.946,0.688,1.0,0.785,0
3,0.467,1,1,1,0.752,0.732,0.101,0.0,0.844,0
4,0.967,0,0,0,0.228,0.266,0.285,0.0,0.632,0
5,0.867,1,0,1,0.625,0.543,0.989,0.0,0.021,0
6,0.267,0,1,2,0.936,0.246,0.015,0.0,0.927,0
7,0.500,0,0,1,0.151,0.035,0.130,0.0,0.465,1
8,0.400,1,0,0,0.513,0.291,0.627,0.0,0.153,0
9,0.900,0,1,0,0.818,0.592,0.536,0.0,0.645,0


In [4]:
#import regression dataset
dfr = pd.read_csv('houses.csv')

#removing columns
dfr = dfr.drop(columns=['renovated', 'quartile_zone'])

#encoding true/false
dfr = dfr.replace({True: 1, False: 0})

#Extracting the year from the data
# Convert 'date' column to datetime format
dfr['date'] = pd.to_datetime(dfr['date'])
# Extract the year and replace the original 'date' column
dfr['date'] = dfr['date'].dt.year

#normalizing data
columns_to_normalize = ["price", "living_in_m2"]
dfr[columns_to_normalize] = dfr[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)


#spliting the dataframe
#training 80% of dataset
#testing 20% of dataset
split_point3 = int(0.8 * len(dfr))
train_regression = dfr[:split_point3]
test_regression = dfr[split_point3:]

train_regression.head(10)

<ipython-input-4-1c407e60d625>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfr = dfr.replace({True: 1, False: 0})


,date,price,bedrooms,grade,has_basement,living_in_m2,nice_view,perfect_condition,real_bathrooms,has_lavatory,single_floor,month
0,2014,0.215,2,2,1,0.261,0,0,2,1,0,5
1,2014,0.213,2,2,0,0.166,0,0,1,1,0,11
2,2014,0.223,2,2,0,0.201,1,0,1,0,1,12
3,2015,0.171,2,3,0,0.299,0,0,2,1,1,2
4,2015,0.576,3,2,1,0.856,0,0,3,0,0,1
5,2015,0.339,1,2,1,0.103,1,0,1,0,1,3
6,2014,0.149,2,2,1,0.253,0,0,1,1,1,6
7,2014,0.143,1,1,0,0.264,0,0,1,0,1,10
8,2014,0.116,2,2,0,0.166,0,0,1,1,1,7
9,2014,0.432,2,3,0,0.204,0,0,2,1,0,9


In [5]:
dfb = pd.read_csv('alzheimer.csv')

# Selecionar colunas úteis
columns_to_keep = [
    'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity',
    'DietQuality', 'SleepQuality', 'Smoking',
    'FamilyHistoryAlzheimers', 'Depression', 'Hypertension', 'Diagnosis'
]

# Filtrar o DataFrame
filtered_alzheimer_data = dfb[columns_to_keep]

# Verificar valores ausentes
missing_values = filtered_alzheimer_data.isnull().sum()

# Preencher valores ausentes (se houver)
filtered_alzheimer_data.fillna(filtered_alzheimer_data.median(), inplace=True)

# Salvar em um novo CSV
cleaned_file_path = 'alzheimer_cleaned.csv'
filtered_alzheimer_data.to_csv(cleaned_file_path, index=False)

missing_values, cleaned_file_path


<ipython-input-5-cd5cf057ebfb>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_alzheimer_data.fillna(filtered_alzheimer_data.median(), inplace=True)


(Age                        0
 BMI                        0
 AlcoholConsumption         0
 PhysicalActivity           0
 DietQuality                0
 SleepQuality               0
 Smoking                    0
 FamilyHistoryAlzheimers    0
 Depression                 0
 Hypertension               0
 Diagnosis                  0
 dtype: int64,
 'alzheimer_cleaned.csv')